<a href="https://colab.research.google.com/github/Sewlell/KohyaColab/blob/main/SD3July12-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is honestly a super bare-foot notebook. You may encounter some errors in the middle of the way. If you encountered one, you can report those to my repository.

In [ ]:
# Follow huggingface_hub's instruction so you can download the SD3.

import huggingface_hub

!huggingface-cli login


In [ ]:
# prompt: access of google drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from huggingface_hub import hf_hub_download

# Define the repository ID and the filename you want to download (i.e. stabilityai/stable-diffusion-3-medium)
### If you want to download a model inside a folder, type it like this "the_folder/pytorch_model_diffusion.safetensors"
repo_id = 'stabilityai/stable-diffusion-3-medium'
filename = 'text_encoders/clip_g.safetensors'

# Download the file
file_path = hf_hub_download(repo_id=repo_id, filename=filename)

print(f"File downloaded to {file_path}")


In [ ]:
import zipfile
import os

# Path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/LORAImage/sd15.-.woman (1).zip'

# Directory to extract the contents to
extract_dir = '/content/dataset/'

os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'Extracted {zip_file_path} to {extract_dir}')


In [ ]:
%cd /content
!git clone -b sd3 https://github.com/kohya-ss/sd-scripts
%cd /content/sd-scripts

!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade -r requirements.txt
# do not restart immediately, restart after this cell is done.
!pip install xformers==0.0.26.post1 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install torchvision==0.18.0 --index-url https://download.pytorch.org/whl/cu118

# **Training**

Hey, you need a dataset_config.toml first before anything. This notebook doesn't creation of that unfortunately at the time. (Still you can add your own here) Checkout this doc from kohya-ss (https://github.com/darkstorm2150/sd-scripts/blob/main/docs/config_README-en.md)

Your dataset must be captioned first as well. You can tag your image automatically with the tagger (https://github.com/jhc13/taggui/)

In [ ]:
!accelerate launch "/content/sd-scripts/sd3_train.py" \
                  --enable_bucket --min_bucket_reso="256" --max_bucket_reso="1024" --bucket_reso_steps=20 \
                  --pretrained_model_name_or_path="/content/pretrained_models/sd3_medium.safetensors" \
                  --train_data_dir="/content/SD++A1/" \
                  --optimizer_type="adafactor" \
                  --learning_rate="1e-6" \
                  --xformers --t5xxl_dtype="fp16" --dataset_repeats="15" --shuffle_caption \
                  --optimizer_args=["scale_parameter=False","relative_step=False","warmup_init=False"] \
                  --vae_batch_size="2" --text_encoder_batch_size="2" \
                  --caption_extension=".txt" --output_name="SD++A1" --save_model_as=safetensors \
                  --save_precision="fp16" --train_batch_size="2" \
                  --lr_scheduler="constant" --resolution="512,512" --save_every_n_epochs="2" --lr_scheduler_num_cycles="20" \
                  --clip_g="/content/pretrained_models/text_encoders/clip_g.safetensors" \
                  --clip_l="/content/pretrained_models/text_encoders/clip_l.safetensors" \
                  --t5xxl="/content/pretrained_models/text_encoders/t5xxl_fp16.safetensors" \
                  --max_train_epochs="4" \
                  --sample_prompts="1girl, solo, bangs, blonde, yellow eyes, short hair, :o, spear, treant, ornament, pants, leather jacket, leather boots, fennec ears, fennec girl, scarf, accessories, medieval, dynamic pose, pose, stance, dynamic stance, masterpiece, best quality, detailed," \
                  --sample_sampler=ddim \
                  --sample_every_n_epochs="2" \
                  --cache_text_encoder_outputs --cache_latents \
                  --output_dir="/content/drive/MyDrive/LORAImage"

Ignore `/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead._torch_pytree._register_pytree_node(` error

`--train_data_dir` change your train_data_dir to your desired folder.\

`--optimizer_type`  original repository reccomended adafactor as `--optimizer_type`, but again you can experimenting your own with dadaption or adamw (adamw8bit with --use_8bit_adam) \

`--learning_rate` knob the learning rate along with the batch size. it could influence the result heavily \

`--caption_extension` keep the `caption_extension` .txt unless your caption is other format. \

`--output_name` is your project name, for `save_model_as` keep it safetensors unless you know what you really wanted. \

`--lr_scheduler` feel free to change lr_scheduler with `constant_with_warmup` or `cosine`, keep the `resolution` to either 512,512 , 756,756 , or 1024,1024 \

change `--sample_prompts` if you dont want my default prompt. you can try to add some command like `--h` (height) `--w` (wide) `--n` (negative prompt) `--s` (steps) `--l` (cfg). Prompt weighting do not worked. \

you got quite a lot of choices here but primarily you can choose default `ddim` and `euler`. DO NOT USE ANCESTRAL VARIANTS (euler_a, dpm_2_ancestral) AND SDE (dpmpp_2m_sde), SD3 DOESN'T COMPATIBLE WITH THOSE FUNDAMENTALLY.

change your `output_dir` and `dataset_config` to your desired folder.

**Run the cell below if you know what you are doing and the errors show above.**

In [ ]:
#@title Changing transformers version.

!pip install transformers==4.28.0

In [ ]:
!accelerate config